### 1. Extract data of Toronto neighborhoods from Wikipedia
First step to importing libraries. bs4 and request are used to extract data from the URL, tabulate then to change the data into a table

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate
import folium

In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
toronto_df = df[0]
toronto_df.head(2)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned


### 2. Data cleaning 
At first, I delete lines that contain "Not assigned". The next step is to group the data, rename the columns and reset the indexes.

In [3]:
toronto_df.drop(index=toronto_df[toronto_df['Borough']=='Not assigned'].index, inplace=True)
toronto_df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(', '.join)
df = pd.DataFrame(toronto_df)
df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df.reset_index(inplace=True)
df = df[['PostalCode', 'Borough', 'Neighborhood']]
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
df.shape

(103, 3)

After cleaning the dataset remain 103 rows and 3 columns

### 3. Add geo coordinates
Unfortunately I wasn't able to get geolocation with Geocoder, so I added geolocation from csv file. 

In [5]:
geo = pd.read_csv('Geospatial_Coordinates.csv')
data = pd.concat([df, geo], axis=1, join='inner')
data.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M1G,43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476


### 4. Data Visualization 

Using the Folium library, a map of Toronto is created, to which indicators (based on longitude and latitude) are then applied.

In [6]:
toronto_map = folium.Map(location=[43.651070, -79.347015], zoom_start=10)

toronto_map

In [7]:
incidents = folium.map.FeatureGroup()

for lat, lng, label in zip(data['Latitude'], data['Longitude'], data['PostalCode']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        popup=label,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(toronto_map)

# show map
toronto_map